In [1]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
# Install pyreadr library
#!pip install pyreadr

# Read the data from .RDS files
import pyreadr
data =  pyreadr.read_r('2019_1min_df.RDS')
df_2019 = data[None]
data = pyreadr.read_r('2020_1min_df.RDS')
df_2020 = data[None]
data = pyreadr.read_r('2021_1min_df.RDS')
df_2021 = data[None]
data = pyreadr.read_r('2022_1min_df.RDS')
df_2022 = data[None]
data = pyreadr.read_r('2023_1min_df.RDS')
df_2023 = data[None]

# Merge all the dataframes to form one dataframe.
df = pd.concat([df_2019, df_2020, df_2021, df_2022, df_2023], ignore_index=True)

df['datetime_utc'] = pd.to_datetime(df['datetime_utc'])

df.shape

(2629440, 10)

In [3]:
# Fill Null Values using foward fill and backward fill
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

# 1. Modeling using Principal Component Analysis

## 1. Rocky Reach

### Feature Engineering

In [4]:
df_rr_lagged = df.copy()

# Rocky Reach Headwater Elevation lag for the last 2 minutes
for lag in range(1, 3):
    df_rr_lagged[f'Rocky_Reach_Headwater_Elevation_lag_{lag}'] = df['Rocky_Reach_Headwater_Elevation'].shift(lag)

# Rocky Reach Discharge lag for the last 2 minutes
for lag in range(1, 3):
    df_rr_lagged[f'Rocky_Reach_Total_Discharge_lag_{lag}'] = df['Rocky_Reach_Total_Discharge'].shift(lag)

# Wells Flow Impact = flows averaged from previous 90-105 minutes
for lag in range(90, 106):
    df_rr_lagged[f'Wells_Total_Discharge_lag_{lag}'] = df['Wells_Total_Discharge'].shift(lag)

# Chelan Flow Impact = flows averaged from previous 62-77 minutes
for lag in range(62, 78):
    df_rr_lagged[f'Chelan_Flow_lag_{lag}'] = df['Chelan_Flow'].shift(lag)

# Entiat Flow Impact = flows averaged from previous 15-30 minutes
for lag in range(15, 31):
    df_rr_lagged[f'Entiat_Flow_lag_{lag}'] = df['Entiat_Flow'].shift(lag)

df_rr_lagged.columns

Index(['datetime_utc', 'datetime_stamp', 'Wells_Total_Discharge',
       'Chelan_Flow', 'Entiat_Flow', 'Rocky_Reach_Total_Discharge',
       'Wenatchee_Flow', 'Rock_Island_Total_Discharge',
       'Rock_Island_Headwater_Elevation', 'Rocky_Reach_Headwater_Elevation',
       'Rocky_Reach_Headwater_Elevation_lag_1',
       'Rocky_Reach_Headwater_Elevation_lag_2',
       'Rocky_Reach_Total_Discharge_lag_1',
       'Rocky_Reach_Total_Discharge_lag_2', 'Wells_Total_Discharge_lag_90',
       'Wells_Total_Discharge_lag_91', 'Wells_Total_Discharge_lag_92',
       'Wells_Total_Discharge_lag_93', 'Wells_Total_Discharge_lag_94',
       'Wells_Total_Discharge_lag_95', 'Wells_Total_Discharge_lag_96',
       'Wells_Total_Discharge_lag_97', 'Wells_Total_Discharge_lag_98',
       'Wells_Total_Discharge_lag_99', 'Wells_Total_Discharge_lag_100',
       'Wells_Total_Discharge_lag_101', 'Wells_Total_Discharge_lag_102',
       'Wells_Total_Discharge_lag_103', 'Wells_Total_Discharge_lag_104',
       'Wells_T

In [5]:
# List of unnecessary columns to drop
columns_to_drop = ['datetime_utc', 'datetime_stamp', 'Wells_Total_Discharge','Chelan_Flow', 'Entiat_Flow', 
                   'Wenatchee_Flow', 'Rock_Island_Total_Discharge', 'Rock_Island_Headwater_Elevation']

# Drop the unnecessary columns
df_rr = df_rr_lagged.drop(columns=columns_to_drop).copy()

# Drop rows with missing values
df_rr.dropna(inplace=True)

df_rr.shape

(2629335, 54)

### PCA

In [6]:
# Separate features and target
target = 'Rocky_Reach_Headwater_Elevation'
X = df_rr.drop(columns=[target]) 
y = df_rr[target]  # Target

**Run the following code to check how the R^2 changes with number of components.**

**The optimal components by running this is 16**

#### Optimal Components - 16 

In [7]:
pca = PCA(n_components=16)
X_pca = pca.fit_transform(X)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, shuffle=False)

# Model training with Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error for Rocky Reach Dam:", mean_squared_error(y_test, y_pred))
print("R-Squared for Rocky Reach Dam:", r2_score(y_test, y_pred))

Mean Squared Error for Rocky Reach Dam: 0.0025159530261372686
R-Squared for Rocky Reach Dam: 0.9942786545677642


In [8]:
# Features from the data
feature_names = X.columns

# Components given by pca
components = pca.components_

# Function to get top features for each component
def get_top_features_for_component(component, num_features):
    sorted_feature_indices = np.argsort(np.abs(component))[::-1][:num_features]
    return feature_names[sorted_feature_indices]

# Getting top 10 features for each component
num_top_features = 10 
top_features_per_component = {}

for i, component in enumerate(components):
    top_features = get_top_features_for_component(component, num_top_features)
    top_features_per_component[f"Component {i+1}"] = top_features

# Counting how often each feature appears in the top lists
feature_counts = pd.Series(np.concatenate(list(top_features_per_component.values()))).value_counts()

# Displaying the most frequently appearing features
print("10 Most Influential Features in PCA with 16 Components:")
print(feature_counts.head(10))

10 Most Influential Features in PCA with 16 Components:
Wells_Total_Discharge_lag_101    10
Wells_Total_Discharge_lag_98     10
Wells_Total_Discharge_lag_97      9
Wells_Total_Discharge_lag_105     9
Wells_Total_Discharge_lag_90      9
Wells_Total_Discharge_lag_102     9
Wells_Total_Discharge_lag_93      9
Wells_Total_Discharge_lag_96      9
Wells_Total_Discharge_lag_94      8
Wells_Total_Discharge_lag_100     8
dtype: int64


## 2. Rock Island

### Feature Engineering

In [9]:
df_ri_lagged = df.copy()

# Rock Island Headwater Elevation lag for last 2 minutes
for lag in range(1, 3):
    df_ri_lagged[f'Rock_Island_Headwater_Elevation_lag_{lag}'] = df['Rock_Island_Headwater_Elevation'].shift(lag)

# Rocky Island Discharge lag for last 2 minutes
for lag in range(1, 3):
    df_ri_lagged[f'Rock_Island_Total_Discharge_lag_{lag}'] = df['Rock_Island_Total_Discharge'].shift(lag)

# Wells Flow Impact = flows from last 15-30 minutes.
for lag in range(15, 31):
    df_ri_lagged[f'Wenatchee_Flow_lag_{lag}'] = df['Wenatchee_Flow'].shift(lag)

# Rocky Reach Discharge Impact = Discharge lags from last 60 to 90 minutes
for lag in range(60, 91):
    df_ri_lagged[f'Rocky_Reach_Total_Discharge_lag_{lag}'] = df['Rocky_Reach_Total_Discharge'].shift(lag)

df_ri_lagged.columns

Index(['datetime_utc', 'datetime_stamp', 'Wells_Total_Discharge',
       'Chelan_Flow', 'Entiat_Flow', 'Rocky_Reach_Total_Discharge',
       'Wenatchee_Flow', 'Rock_Island_Total_Discharge',
       'Rock_Island_Headwater_Elevation', 'Rocky_Reach_Headwater_Elevation',
       'Rock_Island_Headwater_Elevation_lag_1',
       'Rock_Island_Headwater_Elevation_lag_2',
       'Rock_Island_Total_Discharge_lag_1',
       'Rock_Island_Total_Discharge_lag_2', 'Wenatchee_Flow_lag_15',
       'Wenatchee_Flow_lag_16', 'Wenatchee_Flow_lag_17',
       'Wenatchee_Flow_lag_18', 'Wenatchee_Flow_lag_19',
       'Wenatchee_Flow_lag_20', 'Wenatchee_Flow_lag_21',
       'Wenatchee_Flow_lag_22', 'Wenatchee_Flow_lag_23',
       'Wenatchee_Flow_lag_24', 'Wenatchee_Flow_lag_25',
       'Wenatchee_Flow_lag_26', 'Wenatchee_Flow_lag_27',
       'Wenatchee_Flow_lag_28', 'Wenatchee_Flow_lag_29',
       'Wenatchee_Flow_lag_30', 'Rocky_Reach_Total_Discharge_lag_60',
       'Rocky_Reach_Total_Discharge_lag_61',
       'Ro

In [10]:
# List of columns to drop
columns_to_drop = ['datetime_utc', 'datetime_stamp', 'Wells_Total_Discharge',
                   'Chelan_Flow', 'Entiat_Flow', 'Rocky_Reach_Total_Discharge',
                   'Wenatchee_Flow', 'Rocky_Reach_Headwater_Elevation']

# Create the DataFrame df_ri by dropping the specified columns
df_ri = df_ri_lagged.drop(columns=columns_to_drop).copy()

# Drop rows with missing values
df_ri.dropna(inplace=True)
df_ri.shape

(2629350, 53)

### PCA

In [11]:
# Separate features and target
target = 'Rock_Island_Headwater_Elevation'
X = df_ri.drop(columns=[target]) 
y = df_ri[target] #Target

**Run the following code to check how the R^2 changes with number of components.**

**The optimal components by running this is 22**

#### Optimal Components - 22

In [12]:
pca = PCA(n_components=22)
X_pca = pca.fit_transform(X)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, shuffle=False)

# Model training with Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error for Rock Island Dam:", mean_squared_error(y_test, y_pred))
print("R-Squared for Rock Island Dam:", r2_score(y_test, y_pred))

Mean Squared Error for Rock Island Dam: 0.0006760730362111756
R-Squared for Rock Island Dam: 0.9969058204878458


In [13]:
# Features from the data
feature_names = X.columns

# Components given by pca
components = pca.components_

# Function to get top features for each component
def get_top_features_for_component(component, num_features):
    sorted_feature_indices = np.argsort(np.abs(component))[::-1][:num_features]
    return feature_names[sorted_feature_indices]

# Getting top 10 features for each component
num_top_features = 10 
top_features_per_component = {}

for i, component in enumerate(components):
    top_features = get_top_features_for_component(component, num_top_features)
    top_features_per_component[f"Component {i+1}"] = top_features

# Counting how often each feature appears in the top lists
feature_counts = pd.Series(np.concatenate(list(top_features_per_component.values()))).value_counts()

# Displaying the most frequently appearing features
print("10 Most Influential Features in PCA of 22 Components")
print(feature_counts.head(10))

10 Most Influential Features in PCA of 22 Components
Rocky_Reach_Total_Discharge_lag_90    11
Rocky_Reach_Total_Discharge_lag_60    10
Rocky_Reach_Total_Discharge_lag_74     9
Rocky_Reach_Total_Discharge_lag_75     9
Rocky_Reach_Total_Discharge_lag_85     8
Rocky_Reach_Total_Discharge_lag_76     8
Rocky_Reach_Total_Discharge_lag_71     8
Rocky_Reach_Total_Discharge_lag_65     8
Rocky_Reach_Total_Discharge_lag_67     8
Rocky_Reach_Total_Discharge_lag_86     7
dtype: int64


# 2. Seasonality Using PCA

## 1. Rocky Reach

### Spring - (3,4,5 months)

In [14]:
# Remove the NA values
df_rr_lagged.dropna(inplace = True)

# Filter for spring months
df_rr_spring = df_rr_lagged[df_rr_lagged['datetime_utc'].dt.month.isin([3, 4, 5])]

# List of unnecessary columns to drop (as before)
columns_to_drop_spring = ['datetime_utc', 'datetime_stamp', 'Wells_Total_Discharge', 'Chelan_Flow', 'Entiat_Flow', 
                          'Wenatchee_Flow', 'Rock_Island_Total_Discharge', 'Rock_Island_Headwater_Elevation']

# Drop the unnecessary columns from df_rr_spring
df_rr_spring = df_rr_spring.drop(columns=columns_to_drop_spring)

# Display the shape of the df_rr_spring DataFrame to confirm the changes
df_rr_spring.shape

(662400, 54)

In [15]:
# Separate features and target
target = 'Rocky_Reach_Headwater_Elevation'
X = df_rr_spring.drop(columns=[target]) 
y = df_rr_spring[target] #Target

**Run the following code to check how the R^2 changes with number of components.**

**The optimal components by running this is 17**

#### Optimal Components - 17

In [16]:
pca = PCA(n_components=17)
X_pca = pca.fit_transform(X)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, shuffle=False)

# Model training with Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error for Rock Island Dam:", mean_squared_error(y_test, y_pred))
print("R-Squared for Rock Island Dam:", r2_score(y_test, y_pred))

Mean Squared Error for Rock Island Dam: 0.0023633777134123937
R-Squared for Rock Island Dam: 0.9950807928984331


In [17]:
# Features from the data
feature_names = X.columns

# Components given by pca
components = pca.components_

# Function to get top features for each component
def get_top_features_for_component(component, num_features):
    sorted_feature_indices = np.argsort(np.abs(component))[::-1][:num_features]
    return feature_names[sorted_feature_indices]

# Getting top 10 features for each component
num_top_features = 10 
top_features_per_component = {}

for i, component in enumerate(components):
    top_features = get_top_features_for_component(component, num_top_features)
    top_features_per_component[f"Component {i+1}"] = top_features

# Counting how often each feature appears in the top lists
feature_counts = pd.Series(np.concatenate(list(top_features_per_component.values()))).value_counts()

# Displaying the most frequently appearing features
print("10 Most Influential Features in PCA of 17 Components from Rocky Reach Spring Data.")
print(feature_counts.head(10))

10 Most Influential Features in PCA of 17 Components from Rocky Reach Spring Data.
Wells_Total_Discharge_lag_101    12
Wells_Total_Discharge_lag_97     11
Wells_Total_Discharge_lag_98     11
Wells_Total_Discharge_lag_95     11
Wells_Total_Discharge_lag_100    10
Wells_Total_Discharge_lag_94     10
Wells_Total_Discharge_lag_90      9
Wells_Total_Discharge_lag_91      8
Wells_Total_Discharge_lag_105     8
Wells_Total_Discharge_lag_93      8
dtype: int64


### Summer - 6,7,8 months

In [18]:
# Remove the NA values
df_rr_lagged.dropna(inplace = True)

df_rr_summer = df_rr_lagged[df_rr_lagged['datetime_utc'].dt.month.isin([6, 7, 8])]

# List of unnecessary columns to drop (as before)
columns_to_drop_summer = ['datetime_utc', 'datetime_stamp', 'Wells_Total_Discharge', 'Chelan_Flow', 'Entiat_Flow', 'Wenatchee_Flow', 'Rock_Island_Total_Discharge', 'Rock_Island_Headwater_Elevation']

# Drop the unnecessary columns from df_rr_spring
df_rr_summer = df_rr_summer.drop(columns=columns_to_drop_summer)

# Display the shape of the df_rr_spring DataFrame to confirm the changes
df_rr_summer.shape

(662400, 54)

In [19]:
# Separate features and target
target = 'Rocky_Reach_Headwater_Elevation'
X = df_rr_summer.drop(columns=[target]) 
y = df_rr_summer[target] #Target

**Run the following code to check how the R^2 changes with number of components.**

**The optimal components by running this is 17**

#### Optimal Components - 17

In [20]:
pca = PCA(n_components=17)
X_pca = pca.fit_transform(X)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, shuffle=False)

# Model training with Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R-Squared for Rock:", r2_score(y_test, y_pred))

Mean Squared Error: 0.0014911463393961295
R-Squared for Rock: 0.996542216990568


In [21]:
# Features from the data
feature_names = X.columns

# Components given by pca
components = pca.components_

# Function to get top features for each component
def get_top_features_for_component(component, num_features):
    sorted_feature_indices = np.argsort(np.abs(component))[::-1][:num_features]
    return feature_names[sorted_feature_indices]

# Getting top 10 features for each component
num_top_features = 10 
top_features_per_component = {}

for i, component in enumerate(components):
    top_features = get_top_features_for_component(component, num_top_features)
    top_features_per_component[f"Component {i+1}"] = top_features

# Counting how often each feature appears in the top lists
feature_counts = pd.Series(np.concatenate(list(top_features_per_component.values()))).value_counts()

# Displaying the most frequently appearing features
print("10 Most Influential Features in PCA of 17 Components from Rocky Reach Summer Data.")
print(feature_counts.head(10))

10 Most Influential Features in PCA of 17 Components from Rocky Reach Summer Data.
Wells_Total_Discharge_lag_98     12
Wells_Total_Discharge_lag_97     11
Wells_Total_Discharge_lag_95     11
Wells_Total_Discharge_lag_101    11
Wells_Total_Discharge_lag_94     11
Wells_Total_Discharge_lag_104     9
Wells_Total_Discharge_lag_100     9
Wells_Total_Discharge_lag_93      9
Wells_Total_Discharge_lag_90      8
Wells_Total_Discharge_lag_105     8
dtype: int64


### Autumn - 9,10,11

In [22]:
# Remove the NA values
df_rr_lagged.dropna(inplace = True)

df_rr_autumn = df_rr_lagged[df_rr_lagged['datetime_utc'].dt.month.isin([9, 10, 11])]

# List of unnecessary columns to drop (as before)
columns_to_drop_autumn = ['datetime_utc', 'datetime_stamp', 'Wells_Total_Discharge', 'Chelan_Flow', 'Entiat_Flow', 'Wenatchee_Flow', 'Rock_Island_Total_Discharge', 'Rock_Island_Headwater_Elevation']

# Drop the unnecessary columns from df_rr_spring
df_rr_autumn = df_rr_autumn.drop(columns=columns_to_drop_autumn)

# Display the shape of the df_rr_spring DataFrame to confirm the changes
df_rr_autumn.shape

(655200, 54)

In [23]:
# Separate features and target
target = 'Rocky_Reach_Headwater_Elevation'
X = df_rr_autumn.drop(columns=[target]) 
y = df_rr_autumn[target] #Target

**Run the following code to check how the R^2 changes with number of components.**

**The optimal components by running this is 14**

#### Optimal Components - 14

In [24]:
pca = PCA(n_components=14)
X_pca = pca.fit_transform(X)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, shuffle=False)

# Model training with Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R-Squared for Rock:", r2_score(y_test, y_pred))

Mean Squared Error: 0.0017459070864256812
R-Squared for Rock: 0.9954831422231587


In [25]:
# Features from the data
feature_names = X.columns

# Components given by pca
components = pca.components_

# Function to get top features for each component
def get_top_features_for_component(component, num_features):
    sorted_feature_indices = np.argsort(np.abs(component))[::-1][:num_features]
    return feature_names[sorted_feature_indices]

# Getting top 10 features for each component
num_top_features = 10 
top_features_per_component = {}

for i, component in enumerate(components):
    top_features = get_top_features_for_component(component, num_top_features)
    top_features_per_component[f"Component {i+1}"] = top_features

# Counting how often each feature appears in the top lists
feature_counts = pd.Series(np.concatenate(list(top_features_per_component.values()))).value_counts()

# Displaying the most frequently appearing features
print("10 Most Influential Features in PCA of 17 Components from Rocky Reach Autumn Data.")
print(feature_counts.head(10))

10 Most Influential Features in PCA of 17 Components from Rocky Reach Autumn Data.
Wells_Total_Discharge_lag_105    10
Wells_Total_Discharge_lag_98      9
Wells_Total_Discharge_lag_100     9
Wells_Total_Discharge_lag_90      9
Wells_Total_Discharge_lag_97      8
Wells_Total_Discharge_lag_96      8
Wells_Total_Discharge_lag_99      8
Wells_Total_Discharge_lag_95      8
Wells_Total_Discharge_lag_101     8
Wells_Total_Discharge_lag_93      8
dtype: int64


### Winter - 12, 1, 2

In [26]:
# Remove the NA values
df_rr_lagged.dropna(inplace = True)

df_rr_winter = df_rr_lagged[df_rr_lagged['datetime_utc'].dt.month.isin([12, 1, 2])]

# List of unnecessary columns to drop (as before)
columns_to_drop_winter = ['datetime_utc', 'datetime_stamp', 'Wells_Total_Discharge', 'Chelan_Flow', 'Entiat_Flow', 'Wenatchee_Flow', 'Rock_Island_Total_Discharge', 'Rock_Island_Headwater_Elevation']

# Drop the unnecessary columns from df_rr_spring
df_rr_winter = df_rr_winter.drop(columns=columns_to_drop_winter)

# Display the shape of the df_rr_spring DataFrame to confirm the changes
df_rr_winter.shape

(649335, 54)

In [27]:
# Separate features and target
target = 'Rocky_Reach_Headwater_Elevation'
X = df_rr_winter.drop(columns=[target]) 
y = df_rr_winter[target] #Target

**Run the following code to check how the R^2 changes with number of components.**

**The optimal components by running this is 14**

#### Optimal Components - 14

In [28]:
pca = PCA(n_components=14)
X_pca = pca.fit_transform(X)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, shuffle=False)

# Model training with Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R-Squared for Rock:", r2_score(y_test, y_pred))

Mean Squared Error: 0.0038518410724965864
R-Squared for Rock: 0.989230551042628


In [29]:
# Features from the data
feature_names = X.columns

# Components given by pca
components = pca.components_

# Function to get top features for each component
def get_top_features_for_component(component, num_features):
    sorted_feature_indices = np.argsort(np.abs(component))[::-1][:num_features]
    return feature_names[sorted_feature_indices]

# Getting top 10 features for each component
num_top_features = 10 
top_features_per_component = {}

for i, component in enumerate(components):
    top_features = get_top_features_for_component(component, num_top_features)
    top_features_per_component[f"Component {i+1}"] = top_features

# Counting how often each feature appears in the top lists
feature_counts = pd.Series(np.concatenate(list(top_features_per_component.values()))).value_counts()

# Displaying the most frequently appearing features
print("10 Most Influential Features in PCA of 17 Components from Rocky Reach Winter Data.")
print(feature_counts.head(10))

10 Most Influential Features in PCA of 17 Components from Rocky Reach Winter Data.
Wells_Total_Discharge_lag_90     10
Wells_Total_Discharge_lag_98      9
Wells_Total_Discharge_lag_95      9
Wells_Total_Discharge_lag_100     9
Wells_Total_Discharge_lag_101     9
Wells_Total_Discharge_lag_105     9
Wells_Total_Discharge_lag_97      8
Wells_Total_Discharge_lag_93      8
Wells_Total_Discharge_lag_96      7
Wells_Total_Discharge_lag_94      7
dtype: int64


## 2. Rock Island

### Spring - (3,4,5 months)

In [30]:
# Remove the NA values
df_ri_lagged.dropna(inplace = True)

# Filter for spring months
df_ri_spring = df_ri_lagged[df_ri_lagged['datetime_utc'].dt.month.isin([3, 4, 5])]

# List of unnecessary columns to drop (as before)
columns_to_drop_spring = ['datetime_utc', 'datetime_stamp', 'Wells_Total_Discharge','Chelan_Flow', 'Entiat_Flow', 
                          'Rocky_Reach_Total_Discharge','Wenatchee_Flow', 'Rocky_Reach_Headwater_Elevation']

# Drop the unnecessary columns from df_rr_spring
df_ri_spring = df_ri_spring.drop(columns=columns_to_drop_spring)

# Display the shape of the df_rr_spring DataFrame to confirm the changes
df_ri_spring.shape

(662400, 53)

In [31]:
# Separate features and target
target = 'Rock_Island_Headwater_Elevation'
X = df_ri_spring.drop(columns=[target]) 
y = df_ri_spring[target] #Target

 Run the following code to check how the R^2 changes with number of components.

The optimal components by running this is 21

#### Optimal Components - 21


In [38]:
pca = PCA(n_components=21)
X_pca = pca.fit_transform(X)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, shuffle=False)

# Model training with Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error for Rock Island Dam:", mean_squared_error(y_test, y_pred))
print("R-Squared for Rock Island Dam:", r2_score(y_test, y_pred))

Mean Squared Error for Rock Island Dam: 0.0016830182074658168
R-Squared for Rock Island Dam: 0.9928433007180619


In [39]:
# Features from the data
feature_names = X.columns

# Components given by pca
components = pca.components_

# Function to get top features for each component
def get_top_features_for_component(component, num_features):
    sorted_feature_indices = np.argsort(np.abs(component))[::-1][:num_features]
    return feature_names[sorted_feature_indices]

# Getting top 10 features for each component
num_top_features = 10 
top_features_per_component = {}

for i, component in enumerate(components):
    top_features = get_top_features_for_component(component, num_top_features)
    top_features_per_component[f"Component {i+1}"] = top_features

# Counting how often each feature appears in the top lists
feature_counts = pd.Series(np.concatenate(list(top_features_per_component.values()))).value_counts()

# Displaying the most frequently appearing features
print("10 Most Influential Features in PCA of 21 Components from Rock Island Spring Data.")
print(feature_counts.head(10))

10 Most Influential Features in PCA of 21 Components from Rock Island Spring Data.
Rocky_Reach_Total_Discharge_lag_60    11
Rocky_Reach_Total_Discharge_lag_90    11
Rocky_Reach_Total_Discharge_lag_75    10
Rocky_Reach_Total_Discharge_lag_71     8
Rocky_Reach_Total_Discharge_lag_79     7
Rocky_Reach_Total_Discharge_lag_84     7
Rocky_Reach_Total_Discharge_lag_68     7
Rocky_Reach_Total_Discharge_lag_66     7
Rocky_Reach_Total_Discharge_lag_86     7
Rock_Island_Total_Discharge_lag_2      6
dtype: int64


## Summer (6, 7, 8) months

In [41]:
# Remove the NA values
df_ri_lagged.dropna(inplace = True)

# Filter for spring months
df_ri_summer = df_ri_lagged[df_ri_lagged['datetime_utc'].dt.month.isin([6, 7, 8])]

# List of unnecessary columns to drop (as before)
columns_to_drop_summer = ['datetime_utc', 'datetime_stamp', 'Wells_Total_Discharge','Chelan_Flow', 'Entiat_Flow', 
                          'Rocky_Reach_Total_Discharge','Wenatchee_Flow', 'Rocky_Reach_Headwater_Elevation']

# Drop the unnecessary columns from df_rr_spring
df_ri_summer = df_ri_summer.drop(columns=columns_to_drop_summer)

# Display the shape of the df_rr_spring DataFrame to confirm the changes
df_ri_summer.shape

(662400, 53)

In [42]:
# Separate features and target
target = 'Rock_Island_Headwater_Elevation'
X = df_ri_summer.drop(columns=[target]) 
y = df_ri_summer[target] #Target

Run the following code to check how the R^2 changes with number of components.

The optimal components by running this is 21

### Optimal Components - 21

In [44]:
pca = PCA(n_components=21)
X_pca = pca.fit_transform(X)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, shuffle=False)

# Model training with Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error for Rock Island Dam:", mean_squared_error(y_test, y_pred))
print("R-Squared for Rock Island Dam:", r2_score(y_test, y_pred))

Mean Squared Error for Rock Island Dam: 0.0004244119797941725
R-Squared for Rock Island Dam: 0.998551024945339


In [46]:
# Features from the data
feature_names = X.columns

# Components given by pca
components = pca.components_

# Function to get top features for each component
def get_top_features_for_component(component, num_features):
    sorted_feature_indices = np.argsort(np.abs(component))[::-1][:num_features]
    return feature_names[sorted_feature_indices]

# Getting top 10 features for each component
num_top_features = 10 
top_features_per_component = {}

for i, component in enumerate(components):
    top_features = get_top_features_for_component(component, num_top_features)
    top_features_per_component[f"Component {i+1}"] = top_features

# Counting how often each feature appears in the top lists
feature_counts = pd.Series(np.concatenate(list(top_features_per_component.values()))).value_counts()

# Displaying the most frequently appearing features
print("10 Most Influential Features in PCA of 21 Components from Rock Island Summer Data.")
print(feature_counts.head(10))

10 Most Influential Features in PCA of 21 Components from Rock Island Summer Data.
Rocky_Reach_Total_Discharge_lag_90    11
Rocky_Reach_Total_Discharge_lag_60    10
Rocky_Reach_Total_Discharge_lag_65     8
Rocky_Reach_Total_Discharge_lag_64     7
Rocky_Reach_Total_Discharge_lag_67     7
Rocky_Reach_Total_Discharge_lag_75     7
Rocky_Reach_Total_Discharge_lag_79     7
Rocky_Reach_Total_Discharge_lag_85     7
Rocky_Reach_Total_Discharge_lag_71     7
Rocky_Reach_Total_Discharge_lag_81     6
dtype: int64


## Autumn (9, 10, 11) months

In [48]:
# Remove the NA values
df_ri_lagged.dropna(inplace = True)

# Filter for spring months
df_ri_autumn = df_ri_lagged[df_ri_lagged['datetime_utc'].dt.month.isin([9, 10, 11])]

# List of unnecessary columns to drop (as before)
columns_to_drop_autumn = ['datetime_utc', 'datetime_stamp', 'Wells_Total_Discharge','Chelan_Flow', 'Entiat_Flow', 
                          'Rocky_Reach_Total_Discharge','Wenatchee_Flow', 'Rocky_Reach_Headwater_Elevation']

# Drop the unnecessary columns from df_rr_spring
df_ri_autumn = df_ri_autumn.drop(columns=columns_to_drop_autumn)

# Display the shape of the df_rr_spring DataFrame to confirm the changes
df_ri_autumn.shape

(655200, 53)

In [49]:
# Separate features and target
target = 'Rock_Island_Headwater_Elevation'
X = df_ri_autumn.drop(columns=[target]) 
y = df_ri_autumn[target] #Target

Run the following code to check how the R^2 changes with number of components.

The optimal components by running this is 25

### Optimal Components - 25

In [52]:
pca = PCA(n_components=25)
X_pca = pca.fit_transform(X)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, shuffle=False)

# Model training with Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error for Rock Island Dam:", mean_squared_error(y_test, y_pred))
print("R-Squared for Rock Island Dam:", r2_score(y_test, y_pred))

Mean Squared Error for Rock Island Dam: 0.0008812591598818558
R-Squared for Rock Island Dam: 0.9945546170925796


In [53]:
# Features from the data
feature_names = X.columns

# Components given by pca
components = pca.components_

# Function to get top features for each component
def get_top_features_for_component(component, num_features):
    sorted_feature_indices = np.argsort(np.abs(component))[::-1][:num_features]
    return feature_names[sorted_feature_indices]

# Getting top 10 features for each component
num_top_features = 10 
top_features_per_component = {}

for i, component in enumerate(components):
    top_features = get_top_features_for_component(component, num_top_features)
    top_features_per_component[f"Component {i+1}"] = top_features

# Counting how often each feature appears in the top lists
feature_counts = pd.Series(np.concatenate(list(top_features_per_component.values()))).value_counts()

# Displaying the most frequently appearing features
print("10 Most Influential Features in PCA of 21 Components from Rock Island Autumn Data.")
print(feature_counts.head(10))

10 Most Influential Features in PCA of 21 Components from Rock Island Autumn Data.
Rocky_Reach_Total_Discharge_lag_75    11
Rocky_Reach_Total_Discharge_lag_60    11
Rocky_Reach_Total_Discharge_lag_90    10
Rocky_Reach_Total_Discharge_lag_71    10
Rocky_Reach_Total_Discharge_lag_70     9
Rocky_Reach_Total_Discharge_lag_67     9
Rocky_Reach_Total_Discharge_lag_84     8
Rocky_Reach_Total_Discharge_lag_79     8
Rocky_Reach_Total_Discharge_lag_87     8
Rocky_Reach_Total_Discharge_lag_74     7
dtype: int64


## Winter (12, 1, 2) months

In [54]:
# Remove the NA values
df_ri_lagged.dropna(inplace = True)

# Filter for spring months
df_ri_winter = df_ri_lagged[df_ri_lagged['datetime_utc'].dt.month.isin([12, 1, 2])]

# List of unnecessary columns to drop (as before)
columns_to_drop_winter = ['datetime_utc', 'datetime_stamp', 'Wells_Total_Discharge','Chelan_Flow', 'Entiat_Flow', 
                          'Rocky_Reach_Total_Discharge','Wenatchee_Flow', 'Rocky_Reach_Headwater_Elevation']

# Drop the unnecessary columns from df_rr_spring
df_ri_winter = df_ri_winter.drop(columns=columns_to_drop_winter)

# Display the shape of the df_rr_spring DataFrame to confirm the changes
df_ri_winter.shape

(649350, 53)

In [55]:
# Separate features and target
target = 'Rock_Island_Headwater_Elevation'
X = df_ri_winter.drop(columns=[target]) 
y = df_ri_winter[target] #Target

Run the following code to check how the R^2 changes with number of components.

The optimal components by running this is 22

## Optimal Components - 22

In [58]:
pca = PCA(n_components=25)
X_pca = pca.fit_transform(X)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, shuffle=False)

# Model training with Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error for Rock Island Dam:", mean_squared_error(y_test, y_pred))
print("R-Squared for Rock Island Dam:", r2_score(y_test, y_pred))

Mean Squared Error for Rock Island Dam: 0.0007003657093615828
R-Squared for Rock Island Dam: 0.994779740643838


In [59]:
# Features from the data
feature_names = X.columns

# Components given by pca
components = pca.components_

# Function to get top features for each component
def get_top_features_for_component(component, num_features):
    sorted_feature_indices = np.argsort(np.abs(component))[::-1][:num_features]
    return feature_names[sorted_feature_indices]

# Getting top 10 features for each component
num_top_features = 10 
top_features_per_component = {}

for i, component in enumerate(components):
    top_features = get_top_features_for_component(component, num_top_features)
    top_features_per_component[f"Component {i+1}"] = top_features

# Counting how often each feature appears in the top lists
feature_counts = pd.Series(np.concatenate(list(top_features_per_component.values()))).value_counts()

# Displaying the most frequently appearing features
print("10 Most Influential Features in PCA of 21 Components from Rock Island winter Data.")
print(feature_counts.head(10))

10 Most Influential Features in PCA of 21 Components from Rock Island winter Data.
Rocky_Reach_Total_Discharge_lag_90    12
Rocky_Reach_Total_Discharge_lag_60    11
Rocky_Reach_Total_Discharge_lag_75    11
Rocky_Reach_Total_Discharge_lag_71     9
Rocky_Reach_Total_Discharge_lag_79     9
Rocky_Reach_Total_Discharge_lag_74     8
Rocky_Reach_Total_Discharge_lag_76     8
Rocky_Reach_Total_Discharge_lag_84     8
Rocky_Reach_Total_Discharge_lag_82     8
Rocky_Reach_Total_Discharge_lag_68     8
dtype: int64
